# 使用领域（私有）数据微调 ChatGLM3

生成带有 epoch 和 timestamp 的模型文件

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINET

In [3]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
# train_data_path = 'data/zhouyi_dataset_handmade.csv'    # 训练数据路径
train_data_path = '../data/class4_data/zhouyi_dataset_20240118_163659.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 16                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

## 数据处理

In [4]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 160
    })
})


In [5]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"], num_examples=5)

,content,summary
0,请解释一下需卦。,需卦是一个占卜卦象，在周易卦象中，它由下卦乾和上卦坎组成。坎象征着云，乾象征着天，云聚于天，形成了等待时机的卦象。这是一个大吉大利的卜问，特别适合涉水渡河。根据《象辞》，君子观此卦象，可以宴饮安乐，待时而动。\n\n需卦的核心哲学是：稳扎稳打，不可冒失行动，观时待变，耐心等待。在事业上，需要审时度势，守中正，不可急进，自信充满，可化险为夷。在经商中，必须充满耐心，创造条件和机会，行事光明磊落，等到时机成熟后必然一帆风顺。在感情方面，亦需慎重，培养感情，以诚实、热情相待，时机成熟后可以有良好的结果。\n\n需卦紧随坤宫游魂卦，预示着踌躇和期待。得此卦者，需要时机尚未成熟，需要耐心等待，急进反会见凶险。
1,讼卦代表什么？,在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。
2,乾卦的主要讲述内容是什么？,在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。\n\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。
3,师卦在周易中怎样表达教育的概念？,在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。
4,蒙卦在周易中代表了哪些方面？,蒙卦是由艮卦（山）下，坎卦（水）上组成的异卦相叠。它代表着通泰，启蒙的意义。在这里，卜者并非是在向幼稚愚昧的人取求，而是幼稚愚昧的人在向卜者求教。第一次卜筮就得到了神灵的指示。然而，如果轻慢不敬地再三卜筮的话，神灵便不会再示警。总的来说，这是一个吉利的卜问。\n\n蒙卦的核心在于山下有泉的形象，寓意着启蒙。君子观此卦象，应当以果敢坚毅的行动来培养自身的品德，像山泉一样果断行动。然而，此卦乃是离宫四世卦，它代表着回还往复、疑惑不前、多忧愁过失，因而属于凶卦。\n\n蒙卦在个人发展、事业经商、求名婚恋等方面的解释不一。在事业方面，表示事业初建，具有启蒙和通达之象，需要勇敢坚毅的行动；而在经商方面，需要务必小心谨慎，树立高尚的商业道德，不可急功近利；求名方面，需要接受良好的基础教育，陶冶情操。整体而言，此卦提示须忍耐待机而动，听取别人意见，方能通达运势。


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}


In [24]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

In [10]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/160 [00:00<?, ? examples/s]

In [11]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}

In [12]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)


## 加载模型

In [13]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True,
                                  revision='b098244')

model.supports_gradient_checkpointing = True  
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [14]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [15]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [16]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


### QLoRA 微调模型

In [17]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 3
output_dir = f"../model/{model_name_or_path}-epoch{train_epochs}-{timestamp}"

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                            # 输出目录
    per_device_train_batch_size=8,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=1,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=10,                                    # 模型保存步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)


In [19]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

In [20]:
trainer.train()

/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,4.672600
2,4.701000
3,4.274100
4,4.404200
5,3.964200
6,3.707500
7,3.542500
8,3.141400
9,3.024900
10,2.758700


/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning

TrainOutput(global_step=60, training_loss=0.9088179655683537, metrics={'train_runtime': 62.5918, 'train_samples_per_second': 7.669, 'train_steps_per_second': 0.959, 'total_flos': 4623664398827520.0, 'train_loss': 0.9088179655683537, 'epoch': 3.0})

In [21]:
trainer.model.save_pretrained(output_dir)

# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [27]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [28]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [29]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

{"q":"请解释一下需卦？","summmary":"需卦是一个占卜卦象，在周易卦象中，它由下卦乾和上卦坎组成。坎象征着云，乾象征着天，云聚于天，形成了等待时机的卦象。这是一个大吉大利的卜问，特别适合涉水渡河。根据《象辞》，君子观此卦象，可以宴饮安乐，待时而动。\n\n需卦的核心哲学是：稳扎稳打，不可冒失行动，观时待变，耐心等待。在事业上，需要审时度势，守中正，不可急进，自信充满，可化险为夷。在经商中，必须充满耐心，创造条件和机会，行事光明磊落，等到时机成熟后必然一帆风顺。在感情方面，亦需慎重，培养感情，以诚实、热情相待，时机成熟后可以有良好的结果。\n\n需卦紧随坤宫游魂卦，预示着踌躇和期待。得此卦者，需要时机尚未成熟，需要耐心等待，急进反会见凶险"}

In [33]:
input_text = "请解释一下需卦？"

In [34]:
response, history = base_model.chat(tokenizer, query=input_text)

In [35]:
print(response)

需卦（英文名：Xu卦）是《易经》卦象之一，由两个阴爻夹一个阳爻构成，象征着水充足、润泽的状态。需卦是“需”卦，表示等待，意味着事物发展需要时间、耐心和适当的等待。需卦具有以下几个特点：

1. 阳爻为待，阴爻为需。阳爻象征着力量、积极、进取，阴爻则代表柔顺、耐心和等待。在需卦中，阳爻和阴爻相互配合，共同表示等待时机的到来。

2. 需卦的卦象是由两个阴爻夹一个阳爻构成，象征着水充足、润泽的状态。水在古代中国哲学中象征着阴柔、柔顺、适应等特质。因此，需卦反映了人们在面对问题、挑战时，需要具备柔顺、耐心、等待的特性，以等待时机的到来。

3. 需卦的意义不仅限于等待，还涉及到事物发展、治理国家、领导民族等方面。在《易经》中，需卦提醒人们在处理事物时要保持耐心、积极进取，同时要注重阴柔、谦逊的特性，才能达到良好的治理效果。

总之，需卦教导我们在面对问题和挑战时，要保持耐心、柔顺、适应的状态，以等待时机的到来。同时，也要注重力量的发挥，积极进取，才能取得成功。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

{"content":"周易中的讼卦含义是什么？","summary":"	在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。

In [36]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

周易中的讼卦（英文名：Ship卦）是《易经》卦象之一，由两个阴爻夹一个阳爻构成，象征着天平，意味着事物在发展过程中会出现争议、争端、诉讼等现象。讼卦具有以下几个特点：

1. 天平：讼卦的卦象是由两个阴爻夹一个阳爻构成，象征着天平。天平代表了公正、平等、公平等特质。在讼卦中，阳爻象征着决断、权威，阴爻则代表公正、公平。

2. 争端：讼卦意味着争端、争议、诉讼等现象。由于天平象征公正、平等，所以在讼卦中，可能会出现争夺公正、平等地位的现象。因此，讼卦提醒人们在处理争端、争议时要注意公正、公平原则，同时要维护权威，才能达到良好的解决效果。

3. 诉讼：讼卦也象征着诉讼、诉讼过程。在《易经》中，诉讼往往被视为解决问题的一种方式。因此，讼卦提醒人们在面对问题时，要考虑诉讼的可能性，并在诉讼过程中注意公正、公平原则。

总之，讼卦教导我们在处理争端、争议、诉讼等现象时，要注意公正、公平原则，并维护权威。同时，也要谨慎对待诉讼，权衡利弊，以达到解决问题的最佳效果。


## 使用微调后的 ChatGLM3-6B

In [37]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250808_153227"

peft_model_path = f"../model/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [38]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [39]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250808_153227）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [40]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么？", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么？

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250808_153227）：
[gMASK]sop 周易中的讼卦是什么？ 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [41]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250808_153227）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


# 使用不同的训练集合

## 读取新的数据集

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [2]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
train_data_path = '../data/class4_data/dev_sft.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 16                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

## 数据处理

In [4]:
import csv
import re

def process_csv_data(input_path, output_filename):
    """
    将特殊格式的CSV文本处理成标准CSV格式
    
    Args:
        input_text (str): 原始文本内容
        output_filename (str): 输出CSV文件名
    """
    
    # 解析原始文本，提取query和summary
    # 使用正则表达式匹配<s>Human: 和 </s><s>Assistant: 之间的内容
    pattern = r'<s>Human:\s*(.*?)\s*</s><s>Assistant:\s*(.*?)\s*</s>'
    # 准备写入CSV的数据
    csv_data = []
    csv_data.append(['content', 'summary'])  # 表头
    with open(input_path, "r", encoding="utf-8") as f:
        input_text = f.read()
        matches = re.findall(pattern, input_text, re.DOTALL)
        # 处理匹配到的数据
        for query, summary in matches:
            # 清理多余的空白字符
            query = query.strip()
            summary = summary.strip()
            csv_data.append([query, summary])
    
    # 写入CSV文件
    with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(csv_data)
    
    print(f"数据已成功处理并保存到 {output_filename}")

In [6]:
train_data_path_out = "../data/class4_data/custom_train_data.csv"
process_csv_data(train_data_path,train_data_path_out)

数据已成功处理并保存到 ../data/class4_data/custom_train_data.csv


In [7]:
train_data_path = train_data_path_out
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 200
    })
})


In [8]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(dataset["train"], num_examples=5)

,content,summary
0,给定下列单词，把它们组合成一首四行的诗歌。单词：爱、春天、花、心。\n爱、春天、花、心。,春天的花儿竞相开，\n我的心情也跟着欢乐摆动着。\n这美好的季节迎接着爱的出现，\n让我们彼此相拥，共享温馨。
1,语言模型自动为给定的电影标题生成一部电影的剧情概述。\n电影标题：阿凡达（Avatar）,在阿凡达中，地球资源开发公司派遣一支科学家队伍前往名为潘多拉的外星球，以获取该行星稀有矿物质“不可思议”和“未来之门”的使用。因为潘多拉的生命密度非常高，以至于无法耕作、建筑等活动，所以派遣了一批名为“阿凡达”的人类外表机械壳体作为原住民的代理人。在一连串的事件之后，代理人和原住民之间的关系出现了变化，代理人发现原住民的深深的文化和生活方式，逐渐地与他们真正融合在一起，并为保护他们的家园而决定与他们合作对抗地球资源开发公司。最终代理人成功保护了原住民的家园，并以此为开端，改变了地球资源开发公司的政策。
2,从给定的一组文本中删除所有的标点符号和连字符。\n一段包含标点符号和连字符的文本。,"可以使用正则表达式来实现该功能。具体操作如下：\n\n假设文本为text：\n\n```\nimport re\n\ntext = """"Hello, world! - How are you today?""""\ntext = re.sub(r'[^\w\s]', '', text)\ntext = text.replace('-', '')\nprint(text)\n```\n\n输出结果为：\n\n```\nHello world How are you today\n```\n\n解释下上述代码的几个关键步骤：\n\n1. 使用正则表达式 [^\w\s] 匹配所有非字母、非数字、非下划线和非空白字符（即标点符号）；\n2. 使用re.sub()将匹配到的标点符号替换为空字符串；\n3. 使用replace()删除连字符。"
3,将以下文本段落中的所有单词的第一个字母都改为大写字母。\ni love to eat pizza on friday nights with my friends.,I Love To Eat Pizza On Friday Nights With My Friends.
4,根据以下说明生成一个短故事。输出完整的故事。\n故事情节：一个失落的外星人在地球上寻找回家的路，得到一个小女孩的帮助，最终回到了家。,在无垠的宇宙中，一个叫做克瑞斯的外星人不幸地迷失了方向，远离了自己的星球。他漂浮在太空中，无助又孤独，渴望能够找到回家的路。\n\n经过一番漫长的旅程，克瑞斯最终降落到了地球上。尽管他没有遇到任何危险，但他却不知道该往哪个方向去找到自己的家。他感到十分失落和绝望，这时，一位可爱的小女孩出现在他的面前。\n\n这个小女孩名叫艾米，是一位热心肠、聪明伶俐的小姑娘。她注意到克瑞斯凝望着周围，迷惘不知所措，于是走近他问他需要帮助吗？\n\n克瑞斯感到非常高兴，他跟艾米慢慢交流，告诉她自己的故事和迷路的原因。艾米听了之后，很快就明白了他的处境，于是她决定帮助他找到回家的路。\n\n艾米带着克瑞斯走了很久，解释了地球各种文化和风俗习惯。在这段时间里，两个人建立了深厚的友谊。艾米也许有点儿年龄小，但她却有一颗坚强、有爱心的心。\n\n最终，艾米和克瑞斯走到了一座高山上。艾米指了指远方，告诉克瑞斯他的家在那儿，然后快乐地与他道别。克瑞斯意识到这里就是他的家，他感到无限的感激和亲切。\n\n那一刻，克瑞斯深深地感悟到，友谊的果实是永远珍贵的，像艾米这样坚定而有爱心的人将永远占据着他的心。他在艾米的陪伴下恢复了回家的方向，带着感激和珍惜之情，向着家的方向踏上了旅程。


In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}


In [12]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [13]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/200 [00:00<?, ? examples/s]

In [14]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}

In [15]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)


In [16]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True,
                                  revision='b098244')

model.supports_gradient_checkpointing = True  
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [17]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [18]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [19]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


# 微调

In [25]:
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 50
output_dir = f"../model/{model_name_or_path}-epoch{train_epochs}-{timestamp}"

In [26]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                            # 输出目录
    per_device_train_batch_size=16,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.5,                                  # 预热比例
    logging_steps=5,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=10,                                    # 模型保存步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)


## 训练

In [27]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

In [28]:
trainer.train()

Step,Training Loss
5,1.292600
10,1.142300
15,1.031400
20,1.036000
25,1.126900
30,0.961500
35,1.088500
40,1.162000
45,1.057900
50,0.974600


/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning

TrainOutput(global_step=650, training_loss=0.16984649715801844, metrics={'train_runtime': 1734.9551, 'train_samples_per_second': 5.764, 'train_steps_per_second': 0.375, 'total_flos': 1.4693003498048717e+17, 'train_loss': 0.16984649715801844, 'epoch': 50.0})

In [29]:
trainer.model.save_pretrained(output_dir)

# 训练后的模型与原模型对比

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "给定下列单词，把它们组合成一首四行的诗歌。单词：爱、春天、花、心。\n爱、春天、花、心。"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

爱在春天里绽放,
花儿绽放着心声,
用心倾听,
感受爱的美好。


# 使用微调后的

In [11]:
from peft import PeftModel, PeftConfig

epochs = 50
# timestamp = "20240118_164514"
timestamp = "20250809_083930"

peft_model_path = f"../model/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=50, automade-dataset(fixed))-{timestamp}"

In [12]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

## 结果对比

In [13]:
base_response, ft_response = compare_chatglm_results("根据以下说明生成一个短故事。输出完整的故事。\n故事情节：一个失落的外星人在地球上寻找回家的路，得到一个小女孩的帮助，最终回到了家。", base_model, qlora_model, training_tag)

问题：根据以下说明生成一个短故事。输出完整的故事。
故事情节：一个失落的外星人在地球上寻找回家的路，得到一个小女孩的帮助，最终回到了家。

原始输出：
在遥远的宇宙中，有一个叫做克瑞斯的外星人。他来自一个叫尼安的星球，那里风景优美，四季如春。然而，一切都不再正常运行了。尼安星球上的生物都消失了，克瑞斯也无法例外。他被一阵未知的力量传送到了地球。

地球是一个充满奇迹和生命之球的星球。然而，对于一个失落的外星人来说，地球可能看起来像是地狱。他不知道该去哪里， only地球上有生物存在。他遇到了一个叫做艾米的小女孩。艾米年纪尚小，却充满了智慧和善良。看到艾米，克瑞斯感到自己回家了。

克瑞斯向艾米介绍了自己的情况，并请求她的帮助。艾米听说了他的事情，不仅没有害怕，反而主动带领他探索这个星球。两个人在一起探索，克瑞斯学会了人类的语言，了解了人类社会的一切规律。他发现，人类虽然让他感到陌生，但他们的灵魂却像是一个个熟悉的朋友。

在探索的过程中，艾米还掉了许多有趣的东西。他们看到了一只会飞的鸟，艾米向克瑞斯介绍这是飞行器；他们看到了一家餐厅，艾米向克瑞斯介绍了里面的美食；在艾米的帮助下，克瑞斯体验了许多从未尝过的事物。这些体验让克瑞斯重新找到了自己的方向，他决定留在艾米身边，一起度过余生。

随着时间的推移，克瑞斯和艾米建立了深厚的关系。在这个充满挑战和危险的世界里，艾米成为了他的支持和依靠。他们共同度过了许多难忘的时刻，而在艾米的帮助下，克瑞斯也找到了回家的路。

最终，克瑞斯回到了他的家。虽然他想念艾米，但他知道她也会为他祝福。他们两个都在这个世界上，彼此都留下了独特的印记。艾米和克瑞斯之间的故事，是一个充满爱、善良和友谊的故事，它将永远被铭记在两人的心中。


微调后（ChatGLM3-6B(Epoch=50, automade-dataset(fixed))-20250809_083930）：
[gMASK]sop 根据以下说明生成一个短故事。输出完整的故事。
故事情节：一个失落的外星人在地球上寻找回家的路，得到一个小女孩的帮助，最终回到了家。 在无垠的宇宙中，一个叫做克瑞斯的外星人不幸地迷失了方向，远离了自己的星球。他漂浮在太空中，无助又孤独，渴望能够找到回家的路。

经过一番漫长的旅程，克瑞斯最终降落到了地球上。尽管他没有遇到任何危险，但他却不知道该往哪个方向去

In [14]:
query = """角色介绍：
Alice：一个年轻的商业顾问，她在一家知名公司工作，专注于帮助客户经营业务。
Bob：一位年长的企业家，拥有多年的商业经验，目前正在寻求Alice的帮助。

请续写他们的对话内容。Bob: 当然，我最近在我的公司遇到了一些挑战，需要你的帮助来解决。Alice: 好的，请告诉我具体情况，我会尽力协助你。Bob: 我们的市场份额在过去几个月里迅速下降，我认为我们需要重新评估我们的市场策略。Alice: 那么您已经开始着手进行市场调研了吗？Bob: 是的，我们已经进行了调研，但是结果并没有很好地指明应该采取的下一步行动。Alice:"""


In [15]:
base_response, ft_response = compare_chatglm_results(query, base_model, qlora_model, training_tag)

问题：角色介绍：
Alice：一个年轻的商业顾问，她在一家知名公司工作，专注于帮助客户经营业务。
Bob：一位年长的企业家，拥有多年的商业经验，目前正在寻求Alice的帮助。

请续写他们的对话内容。Bob: 当然，我最近在我的公司遇到了一些挑战，需要你的帮助来解决。Alice: 好的，请告诉我具体情况，我会尽力协助你。Bob: 我们的市场份额在过去几个月里迅速下降，我认为我们需要重新评估我们的市场策略。Alice: 那么您已经开始着手进行市场调研了吗？Bob: 是的，我们已经进行了调研，但是结果并没有很好地指明应该采取的下一步行动。Alice:

原始输出：
您可以告诉我您的想法和目标，我会尽力为您提供建议。Bob: 我们需要一种更具有竞争力优势的策略，能够吸引更多的客户。


微调后（ChatGLM3-6B(Epoch=50, automade-dataset(fixed))-20250809_083930）：
[gMASK]sop 角色介绍：
Alice：一个年轻的商业顾问，她在一家知名公司工作，专注于帮助客户经营业务。
Bob：一位年长的企业家，拥有多年的商业经验，目前正在寻求Alice的帮助。

请续写他们的对话内容。Bob: 当然，我最近在我的公司遇到了一些挑战，需要你的帮助来解决。Alice: 好的，请告诉我具体情况，我会尽力协助你。Bob: 我们的市场份额在过去几个月里迅速下降，我认为我们需要重新评估我们的市场策略。Alice: 那么您已经开始着手进行市场调研了吗？Bob: 是的，我们已经进行了调研，但是结果并没有很好地指明应该采取的下一步行动。Alice: 我可以为您提供一些建议。首先，我们可以分析您的目标市场，了解他们的需求和行为。其次，我们可以重新评估我们的产品定位，并考虑在市场上展开更有竞争力的推广活动。最后，我建议我们与您的现有客户进行更深入的交流，以了解他们的反馈和建议。这些措施将有助于您重新建立您的市场份额，并更好地满足您的客户需求。Bob: 您的建议很有道理，谢谢您的帮助。我们会按照您的建议行动。
